<div class="alert alert-block alert-success">
    <p><b>IRONHACK - DATA ANALYSIS FT - JAN22</b></p>
    <i><b>PROJECT:</b> Data cleaning and manipulation - Shark Attack</i>
</div>

---

<div class="alert alert-block alert-info">
    <p><b>Import Libs:</b></p>
</div>

In [23]:
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns
import matplotlib as plt
import re
import time

<div class="alert alert-block alert-info">
<b>Read CSV and identify encodig:</b></div>

In [24]:
shark_data = pd.read_csv("data/attacks.csv", encoding="latin1")
#DataSet documentation: https://www.sharkattackfile.net/

<div class="alert alert-block alert-success">
<b>DataSet Preview:</b></div>

In [25]:
shark_data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


<div class="alert alert-block alert-success">
<b>Data cleaning and manipulation.</b></div>

In [ ]:
<div class="alert alert-block alert-warning">
<p> Identify data types</p></div>

<div class="alert alert-block alert-danger">
<b>Data cleaning and manipulation</b> Identify data types, droppable columns, blank cells.</div>

In [22]:
#check data types
shark_data.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object

#Where should our company install its headquarters at?
#what would be our company's business purpose? Should we sell a product (i.e. Shark repelent), provide a service to warn or
#educate bathers/swimmers
#What would be our target audience? (Government, customers (male, female, does not matter?), ecc)
##What countries have the most shark attacks
##What countries have the deadliest shark attacks
##Shark attacks progression through years
#What season of the year are shark attacks more prevalent
#Shark attacks per country filtered by gender - Should women or men wear any kind of protective device/shark repelent?

In [ ]:
[shark_data["Country"]]